# Rate equations for Gas Laser

In [1]:
using PyPlot
using Distributions
using Calculus
using Cubature
using Interact

  likely near /Users/Fan/.julia/v0.4/Interact/src/IJulia/setup.jl:153
  likely near /Users/Fan/.julia/v0.4/Interact/src/IJulia/setup.jl:157



### constant parameters:

In [2]:
const radius = 0.25; # radius is in cm
const L = 15; # cm

const h = 6.626068e-34; # in Js
const c = 3e8; # in m/s
const ev = 1.60217646e-19; # in J
const kB = 8.617342e-5; # in eV/K
const T = 300; # in K
const kBT = kB*T*8065.73; # in cm-1
const M = 35; # molecular mass in AMU
const norm_time = 1e6; # normalizing to microsec

## The following constants can be found in p189 Henry's dissetation. 
const σ_GKC = 44; # in angstrom^2. Gas Kinetic Collision cross section.
const σ_DD = 320; # in angstrom^2. Dipole-dipole cross section.
const σ_SPT = 137; # in angstrom^2. SPT cross section.
const σ_36 = 1.61; # in angstrom^2. v3->v6 cross section.
# σ_36 is obtained by fitting the result to the experimental result.
const σ_VS = 21; # in angstrom^2. V-swap cross section.

# define kvs (bimolecular collisions); neglected firstly;
const v = 205*sqrt(T/M); # in m/sec. p5 Henry's dissertation;
const kvs = v*σ_VS *(1e-10)^2/norm_time; # in m^3/microsec
# const kvs = 0;
# oscillation energy levels:
const EG = 0;
const E3 = 1050;
const E6 = 1200;
const E23 = 2100;
const E36 = 2250;
const E26 = 2400;

# Boltzmann distribution:
const Q = exp(EG) + 
          exp(-E3/kBT) + 
          2*exp(-E6/kBT) + 
          exp(-E23/kBT) + 
          exp(-E36/kBT) + 
          exp(-E26/kBT);
const f_G = exp(EG)/Q;
const f_3 = exp(-E3/kBT)/Q;
const f_6 = 2*exp(-E6/kBT)/Q; # factor of 2: double degeneracy of V_6
const f_23 = exp(-E23/kBT)/Q;
const f_36 = exp(-E36/kBT)/Q;
const f_26 = exp(-E26/kBT)/Q;

const C3L = 0.005926302;  # degeneracy included. Feb 7th 2015 email from Henry.
const C4L = 0.007374597;
const C5L = 0.008652703;
const C4U = C4L;
const C5U = C5L;

const g_L = 9.0;
const g_U = 11.0;

const NA = 6.0221413e23;

##################### frequency constants ############################
const f₀ = 31.042748176e12; # in Hz. Data from Laser Line Number.pptx
const f_offset = 30e6;
const f_pump = f₀ + f_offset;
const f_dir_lasing = 245.38e9;
const f_ref_lasing = 248.56e9;
const Δ_f₀D = 3.58e-7*f₀*sqrt(T/M); # in Hz. p26 Henry's dissetation;

const f_range = 5*Δ_f₀D;

const n_rot = 18;
const n_vib = 12;

const mu0 = 4e-7*pi
const eps0 = 8.85e-12
const mode_num = 1 # 1: TE01 / 2: TE12 / 3: TE02 / 4: TE22 / 5: TE11 / 6: TE21 / 7: TM01 / 8: TM11
const p_library = [3.83, 5.33, 7.02, 6.71, 1.84, 3.05, 2.4, 3.83] #zeros of Bessel functions.
const n0 = 1.0 # refractive index
const t_spont = 100
const Δν_THz = 25e6 ;

## define parameter type

In [3]:
type parameter
    pressure::Real
    power::Real
    num_layers::Int64
    
    ntotal::Real
    k63::Real
    k36::Real
    k3623::Real
    k2336::Real
    k2636::Real
    k3626::Real
    kro::Real

    Δ_fP::Real
    Δ_f_Rabi::Real
    Δ_f_NT::Real
    
    num_freq::Int64
    df::Real
    f_dist_end::Array
    f_dist_ctr::Array
    velocity::Array
    f_dist_dir_lasing::Array
    gauss_dist::Array
    SHB::Array
    fp_lasing::Array
    pumpR::Array

    Δr::Real
    r_int::Array

    kwall::Array
    
    MFP::Real

    D::Real

    k98_G::Real
    k87_G::Real
    k76_G::Real
    k65_G::Real
    k54_G::Real
    k43_G::Real
    k32_G::Real
    k21_G::Real
    
    k89_G::Real
    k78_G::Real
    k67_G::Real
    k56_G::Real
    k45_G::Real
    k34_G::Real
    k23_G::Real
    k12_G::Real

    k98_3::Real
    k87_3::Real
    k76_3::Real
    k65_3::Real
    k54_3::Real
    k43_3::Real
    k32_3::Real
    k21_3::Real
    
    k89_3::Real
    k78_3::Real
    k67_3::Real
    k56_3::Real
    k45_3::Real
    k34_3::Real
    k23_3::Real
    k12_3::Real
    
    k1a::Real
    k2a::Real
    k3a::Real
    k4a::Real
    k5a::Real
    k6a::Real
    k7a::Real
    k8a::Real
    k9a::Real
    k10a::Real
    k11a::Real
    k12a::Real
    k13a::Real
    k14a::Real
    k15a::Real
    k16a::Real
    k17a::Real
    k18a::Real

    niter::Int64
end

 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in require at ./loading.jl:243
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /Users/Fan/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /Users/Fan/.julia/v0.4/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:447
while loading /Users/Fan/.julia/v0.4/Interact/src/IJulia/statedict.jl, in expression starting on line 1


### wall rate functions

In [157]:
include("wallrates.jl")
include("Lorentz_distribution.jl")
include("Q_select.jl")
include("OPFIR_compute_parameters.jl")
include("OPFIR_compute_rhs.jl")
include("OPFIR_compute_row_col_val.jl")
include("OPFIR_fixedpoint.jl")
include("anderson_accel.jl")
include("OPFIR_func.jl")
include("post_plotting.jl")
include("post_gain.jl")
include("population_inv.jl")
include("Sherman_Morrison.jl")

SM_solve (generic function with 1 method)

In [158]:
pressure_list = [200]; # in mTorr;
power = 10;
num_layer = 50;
U_gain = zeros(size(pressure_list))
for i in 1:length(pressure_list)
    pressure = pressure_list[i]
    sol = OPFIR_func(pressure, power, num_layer, 30);
    U_gain[i] = OPFIR_gain(sol[1], sol[2])
end
println(U_gain)
# post_plotting(sol[1], sol[2], sol[1].pressure, sol[1].power, 10);

matrix size is 391200
norm of sol diff = 1.0
norm of sol diff = 2.6404881642825196e-5
norm of sol diff = 3.3412088603850445e-6
norm of sol diff = 1.0162388194858507e-7
[6.087675652272329e-5]


In [151]:
a3 = lufact(A0)

UMFPACK LU Factorization of a 4-by-4 sparse matrix
Ptr{Void} @0x00007f8040678a40


In [152]:
a3\b

4x1 Array{Float64,2}:
 1.26911 
 1.0202  
 0.719174
 1.07513 

In [ ]:
include("population_inv.jl")
inv = zeros(sol[1].num_layers,1)
for ri in 1:sol[1].num_layers
    inv_U = get_inv_U(sol[1], sol[2], ri) # inversion as a function of velocity
    inv[ri] = sum(inv_U .* sol[1].fp_lasing)
end
plot(inv)


In [ ]:
plot(sol[1].fp_lasing)

In [ ]:
include("post_plotting.jl")
figure(figsize=(16,10))
post_plotting(sol[1], sol[2], sol[1].pressure, sol[1].power, 30)

In [ ]:
include("Wallrates.jl")
WallRate(radius, pressure, sol[1].r_int, sol[1].ntotal)

In [ ]:
sub_sum = zeros(sol[1].num_layers,1);
solution = sol[2];
ri = sol[1].r_int;
for i = 1:sol[1].num_layers
    index1 = (18*sol[1].num_freq + n_vib) * (i-1) + 1;
    index2 = (18*sol[1].num_freq + n_vib) * i;
    sub_sum[i] = sum(solution[index1:index2]) * ri[i];
end
sum((sub_sum))

In [ ]:
include("post_plotting.jl")
include("population_inv.jl")
figure(figsize=(6,4))
hold
plot_inv_U(sol[1], sol[2], 30)
plot_inv_U(sol[1], sol[2], 1)
plot_f0()
legend(["close to the wall", "away from the wall"], fontsize=10)

figure(figsize=(6,4))
hold
plot_inv_L(sol[1], sol[2], 30)
plot_inv_L(sol[1], sol[2], 1)
plot_f0()
legend(["close to the wall", "away from the wall"], fontsize=10)

In [ ]:
include("population_inv.jl")
# 12 is U and 2 is L
# plot(get_Nu_vs_freq_layer(sol[1], sol[2], 30))
# hold
# plot(get_rot_vs_freq_layer(sol[1], sol[2], 11, 30))
freq = sol[1].f_dist_ctr;
figure(figsize=(6,4))
title("with bi-molecular collision, pressure = $pressure")
plot([f₀, f₀], [0, 1e15], "r--")
hold
plot([f_pump, f_pump], [0, 1e15], "r--")
plot(freq, get_inv_U(sol[1], sol[2], 30))
hold
plot(freq, get_inv_U(sol[1], sol[2], 1))
figure(figsize=(6,4))
plot([f₀, f₀], [0, 1e15], "r--")
hold
plot([f_pump, f_pump], [0, 1e15], "r--")
plot(freq, get_inv_L(sol[1], sol[2], 30))
hold
plot(freq, get_inv_L(sol[1], sol[2], 1))

In [ ]:
include("population_inv.jl")
# 12 is U and 2 is L
# plot(get_Nu_vs_freq_layer(sol[1], sol[2], 30))
# hold
# plot(get_rot_vs_freq_layer(sol[1], sol[2], 11, 30))
freq = sol[1].f_dist_ctr;
figure(figsize=(6,4))
title("with bi-molecular collision")
plot([f₀, f₀], [0, 1e15], "r--")
hold
plot([f_pump, f_pump], [0, 1e15], "r--")
plot(freq, get_inv_U(sol[1], sol[2], 30))
hold
plot(freq, get_inv_U(sol[1], sol[2], 1))
figure(figsize=(6,4))
plot([f₀, f₀], [0, 1e15], "r--")
hold
plot([f_pump, f_pump], [0, 1e15], "r--")
plot(freq, get_inv_L(sol[1], sol[2], 30))
hold
plot(freq, get_inv_L(sol[1], sol[2], 1))

In [ ]:
include("population_inv.jl")

In [ ]:
plot(get_inv_L(sol[1], sol[2], 30))
hold
plot(get_inv_L(sol[1], sol[2], 1))

In [ ]:
sol[1].kwall

In [ ]:
include("post_plotting.jl")
figure(figsize=(6,4))
plot_thermalpool_layer(sol[1], sol[2], sol[1].pressure, sol[1].power, 30)
# figure(figsize=(6,4))
# plot_U_vs_freq_layer(sol[1], sol[2], sol[1].pressure, sol[1].power, 30)

In [ ]:
include("OPFIR_compute_parameters.jl")
para = OPFIR_compute_parameters(200, 10, 2);

In [ ]:
layer_unknown = n_rot*para.num_freq + n_vib
sol_0 = zeros(para.num_layers * layer_unknown);
    
max_ele = para.num_freq * para.num_layers * (n_rot*(n_rot+2) + n_vib*(n_rot+n_vib+2))


rowind = ones(Int64,max_ele)
colind = ones(Int64,max_ele)
value = zeros(max_ele)
rhs = zeros(para.num_layers*layer_unknown);
OPFIR_compute_row_col_val(rowind, colind, value, para, sol_0)

matrix = sparse(rowind, colind, value)

for j = 1:size(matrix,1)
        matrix[1,j] = 0;
        matrix[end,j] = 0;
end

for j = 1:para.num_layers
    index1::Int64 = layer_unknown * (j-1) + 1
    index2::Int64 = layer_unknown * j
    matrix[1, index1:index2] = para.r_int[j]
    ind_0E::Int64 = layer_unknown * j - n_vib + 7;
    ind_3E::Int64 = layer_unknown * j - n_vib + 8;
    matrix[end, ind_0E] = para.r_int[j]
    matrix[end, ind_3E] = -para.r_int[j]
end
;

In [ ]:
sol[1].num_freq

In [121]:
A0 = sparse([1, 1, 2, 3, 4], [1, 2, 2, 3,4], [0.527264,  0.234496,  0.882686,  0.88042, 0.5])
b = rand(4,1)

4x1 Array{Float64,2}:
 0.908386
 0.900513
 0.633175
 0.537565

In [86]:
full(A0)

4x4 Array{Float64,2}:
 0.527264  0.234496  0.0      0.0
 0.0       0.882686  0.0      0.0
 0.0       0.0       0.88042  0.0
 0.0       0.0       0.0      0.5

In [53]:
n = size(A0, 1)
rank = 1
row_ind = 3
A1 = A0
if rank==1
    u = zeros(n,1)
    u[row_ind] = 1.0
    vv = A0[row_ind, :]
    vv[row_ind] -= 1.0
    A1[row_ind, :] = 0.0
    A1[row_ind, row_ind] = 1.0
end

1.0

In [54]:
issparse(A1)

true

In [57]:
u*vv + A1

4x4 Array{Float64,2}:
 0.527264  0.234496  0.0  0.0    
 0.0       0.882686  0.0  0.0    
 0.0       0.0       0.0  0.88042
 0.0       0.0       0.5  0.0    

In [84]:
full(A0)

4x4 Array{Float64,2}:
 0.527264  0.234496  0.0  0.0
 0.0       0.882686  0.0  0.0
 0.0       0.0       1.0  0.0
 0.0       0.0       0.0  0.5

In [139]:
A0\b

4x1 Array{Float64,2}:
 1.26911 
 1.0202  
 0.719174
 1.07513 

In [138]:
include("Sherman_Morrison.jl")
SM_solve(A0, 2, [2,1], b)

4x1 Array{Float64,2}:
 1.26911 
 1.0202  
 0.719174
 1.07513 

In [130]:
full(A0)

4x4 Array{Float64,2}:
 0.527264  0.234496  0.0      0.0
 0.0       0.882686  0.0      0.0
 0.0       0.0       0.88042  0.0
 0.0       0.0       0.0      0.5